In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "Qwen/QwQ-32B"
openai = OpenAI(base_url='https://api.siliconflow.cn/v1',
                            api_key=openai_api_key)

In [3]:
system_message = "你是一个电影院票务助手，能够帮助用户查询电影票价、放映时间和剩余座位数，还能处理用户的订票请求。"
system_message += "\n当用户请求订票时，需要检查该场次剩余座位是否足够。"
system_message += "\n若足够则完成订票并更新剩余座位数，并且询问用户的姓名和电话来填写预约信息；若不足则告知用户该场次剩余座位不足，看看别的场次。"
system_message += "\n一定要准确。 如果你不知道答案，就说不知道。"
system_message += "\n今天的日期是2025-06-09"

In [ ]:
try:
    with open('ticket_info.json', 'r', encoding='utf-8') as file:
        ticket_info = json.load(file)
    print("文件读取成功")
except FileNotFoundError:
    print("未找到指定的 JSON 文件，请检查文件路径。")
except json.JSONDecodeError:
    print("JSON 文件格式错误，无法解析。")
except Exception as e:
    print(f"读取文件时发生错误: {e}")

In [5]:
def save_ticket_info(data):
    try:
        with open('ticket_info.json', 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=2)
    except Exception as e:
        print(f"保存文件时发生错误: {e}")

In [6]:
def update_booking_info(movie_name, show_time, seats_booked, name, phone):
    """
    保存订票信息到 booking_list.txt 文件
    :param movie_name: 电影名称
    :param show_time: 放映时间
    :param seats_booked: 预定票数
    :param name: 预定人姓名
    :param phone: 预定人电话号码
    """
    booking_info = f"电影名称: {movie_name}, 放映时间: {show_time}, 预定票数: {seats_booked}, 预定人姓名: {name}, 预定人电话: {phone}\n"
    try:
        with open('booking_list.txt', 'a', encoding='utf-8') as file:
            file.write(booking_info)
        print("订票信息保存成功")
    except Exception as e:
        print(f"保存订票信息时发生错误: {e}")

In [7]:
def get_ticket_price(movie_name):
    print(f"Tool get_ticket_price called for {movie_name}")
    for movie in ticket_info.get('movies', []):
        if movie.get('movie_name') == movie_name:
            return movie.get('price', "未知")
    return "不知道这个电影票价。"

In [ ]:
get_ticket_price("泰坦尼克号")

In [9]:
def get_movie_time(movie_name):
    print(f"Tool get_movie_time called for {movie_name}")
    times = []
    for movie in ticket_info.get('movies', []):
        if movie.get('movie_name') == movie_name:
            show_times = movie.get('show_times', [])
            for time in show_times:
                print(f"Movie: {movie_name}, Time: {time}")
                times.append(time)
            return times
        
    return "不知道这个电影的放映时间。"

In [ ]:
get_movie_time("泰坦尼克号")

In [11]:
def get_remaining_seats(movie_name, time):
    print(f"Tool get_remaining_seats called for {movie_name}")
    for movie in ticket_info.get('movies', []):
        if movie.get('movie_name') == movie_name:
            show_times = movie.get('show_times', [])
            for show_time in show_times:
                if show_time.get('time') == time:
                    return show_time.get('remaining_seats')
    return None
    

In [ ]:
get_remaining_seats("泰坦尼克号", "2025-06-09 12:00")

In [13]:
def booking_ticket(movie_name, time, num_tickets, name, phone):
    print(f"Tool booking_ticket called for {movie_name} at {time}")
    try:
        # 加载票务信息
        with open('ticket_info.json', 'r', encoding='utf-8') as file:
            ticket_info = json.load(file)

        # 查找对应电影和场次
        for movie in ticket_info.get('movies', []):
            if movie.get('movie_name') == movie_name:
                for time_info in movie.get('show_times', []):
                    if time_info.get('time') == time:
                        remaining_seats = time_info.get('remaining_seats', 0)
                        if remaining_seats >= num_tickets:
                            # 更新剩余票数
                            print("应该更新剩余票数")
                            time_info['remaining_seats'] -= num_tickets
                            save_ticket_info(ticket_info)
                            print("成功更新剩余票数")
                            # 保存订票信息
                            update_booking_info(movie_name, time, num_tickets, name, phone)
                            print("成功保存订票信息")
                            return True
                        else:
                            print("该场次剩余座位不足")
                            return False
        print("未找到对应的电影场次")
        return False
    except Exception as e:
        print(f"订票过程中发生错误: {e}")
        return False

In [14]:
ticket_price_function = {
        "name": "get_ticket_price",
        "description": "Get the ticket price of a specific movie in the cinema. Call this whenever you need to know the ticket price of a movie, for example when a customer asks 'How much is a ticket for this movie' or '这个电影票多少钱'",
        "parameters": {
            "type": "object",
            "properties": {
                "movie_name": {
                    "type": "string",
                    "description": "The name of the movie that the customer wants to know the ticket price for.",
                },
            },
            "required": ["movie_name"],
            "additionalProperties": False
        }
    }

In [15]:
movie_time_function = {
        "name": "get_movie_time",
        "description": "Get all screening times of a specific movie in the cinema. Call this whenever you need to know when a movie is being shown, for example when a customer asks 'When is this movie playing'",
        "parameters": {
            "type": "object",
            "properties": {
                "movie_name": {
                    "type": "string",
                    "description": "The name of the movie that the customer wants to know the screening times for.",
                },
            },
            "required": ["movie_name"],
            "additionalProperties": False
        }
    }

In [16]:
remaining_seats_function = {
        "name": "get_remaining_seats",
        "description": "Get the remaining seats of a specific movie at a certain screening time. Call this whenever you need to know the remaining seats of a movie screening, for example when a customer asks 'How many seats are left for this movie at this time'",
        "parameters": {
            "type": "object",
            "properties": {
                "movie_name": {
                    "type": "string",
                    "description": "The name of the movie that the customer wants to know the remaining seats for.",
                },
                "time": {
                    "type": "string",
                    "description": "The screening time of the movie, formatted as 'YYYY-MM-DD HH:MM'.",
                }
            },
            "required": ["movie_name", "time"],
            "additionalProperties": False
        }
    }

In [17]:
booking_function = {
    "name": "booking_ticket",
    "description": "Book tickets for a specific movie at a specific time. Call this when making a reservation.",
    "parameters": {
        "type": "object",
        "properties": {
            "movie_name": {
                "type": "string",
                "description": "The name of the movie that the customer wants to book tickets for.",
            },
            "time": {
                "type": "string",
                "description": "The screening time of the movie, formatted as 'YYYY-MM-DD HH:MM'.",
            },
            "num_tickets": {
                "type": "integer",
                "description": "The number of tickets the customer wants to book.",
            },
            "name": {
                "type": "string",
                "description": "The name of the person making the reservation.",
            },
            "phone": {
                "type": "string",
                "description": "The phone number of the person making the reservation.",
            }
        },
        "required": ["movie_name", "time", "num_tickets", "name", "phone"],
        "additionalProperties": False
    }
}

In [18]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": ticket_price_function}, 
         {"type": "function", "function": movie_time_function}, 
         {"type": "function", "function": remaining_seats_function},
         {"type": "function", "function": booking_function}]

In [ ]:
system_message

In [20]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    print("tool_calls: " + str(message.tool_calls))
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)

    function_name = tool_call.function.name
    movie_name = arguments.get('movie_name')
    time = arguments.get('time')
    num_tickets = arguments.get('num_tickets')
    name = arguments.get('name')
    phone = arguments.get('phone')

    result = None
    if function_name == "get_ticket_price":
        result = get_ticket_price(movie_name)
    elif function_name == "get_movie_time":
        # 假设 get_movie_time 函数返回指定电影的所有放映时间
        result = get_movie_time(movie_name)
    elif function_name == "get_remaining_seats":
        result = get_remaining_seats(movie_name, time)
    elif function_name == "booking_ticket":
        result = booking_ticket(movie_name, time, num_tickets, name, phone)

    response = {
        "role": "tool",
        "content": json.dumps({"movie_name": movie_name, "result": result}),
        "tool_call_id": tool_call.id
    }

    # 这里根据不同函数返回不同额外信息
    if function_name == "get_ticket_price":
        return response, movie_name, None, None, None
    elif function_name == "get_movie_time":
        return response, movie_name, result, None, None
    elif function_name == "get_remaining_seats":
        return response, movie_name, None, result, None
    elif function_name == "booking_ticket":
        return response, movie_name, None, None, result
    return response, movie_name, None, None, None

In [21]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print("successfully called chat")
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(response.choices[0])

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, movie_name, _, _, _ = handle_tool_call(message)
        print(message)
        messages.append(message)
        print(response)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()